# AutoEncoders on Fashion MNIST

**The notebook has been adapted from the notebook provided in David Foster's Generative Deep Learning, 2nd Edition.**

- Book: [Amazon](https://www.amazon.com/Generative-Deep-Learning-Teaching-Machines/dp/1098134184/ref=sr_1_1?keywords=generative+deep+learning%2C+2nd+edition&qid=1684708209&sprefix=generative+de%2Caps%2C93&sr=8-1)
- Original notebook (tensorflow and keras): [Github](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/03_vae/01_autoencoder/autoencoder.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

import jax
import jax.numpy as jnp

from flax import linen as nn
from flax.training import train_state
import optax

## 0. Train parameters

In [11]:
IMAGE_SIZE = 32
CHANNELS = 1
BATCH_SIZE = 128
EMBEDDING_DIM = 2
EPOCHS = 5
LEARNING_RATE = 1e-3

## 1. Prepare dataset

In [12]:
# Process the image file
def preprocess(img):
    img = tf.cast(img, tf.float32) / 255.
    img = tf.pad(img, ((2, 2), (2, 2), (0, 0)), constant_values=0)
    return img

# get MNIST dataset
def get_dataset(num_epochs, batch_size):
    # Download MNIST dataset
    train_ds = tfds.load('mnist', split='train')
    test_ds = tfds.load('mnist', split='test')

    # Mapping images as tf tensors
    train_ds = train_ds.map(lambda sample: {'image': preprocess(sample['image']),
                                            'label': sample['label']})
    test_ds = test_ds.map(lambda sample: {'image': preprocess(sample['image']),
                                          'label': sample['label']})

    # TF dataloader
    train_ds = train_ds.repeat(num_epochs).shuffle(1024)
    train_ds = train_ds.batch(batch_size, drop_remainder=True).prefetch(1)
    test_ds = test_ds.shuffle(1024)
    test_ds = test_ds.batch(batch_size, drop_remainder=True).prefetch(1)

    return train_ds, test_ds

In [13]:
check_train, chech_test = get_dataset(EPOCHS, BATCH_SIZE)
print(next(iter(check_train))['image'].shape)

(128, 32, 32, 1)


## 2. Build the AutoEncoder

In [14]:
# Encoder
class Encoder(nn.Module):
    latents: int
    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=32, kernel_size=(3, 3), strides=2)(x)
        x = nn.relu(x)
        x = nn.Conv(features=64, kernel_size=(3, 3), strides=2)(x)
        x = nn.relu(x)
        x = nn.Conv(features=128, kernel_size=(3, 3), strides=2)(x)
        x = nn.relu(x)
        x = x.reshape(x.shape[0], -1)
        x = nn.Dense(features=self.latents)(x)
        return x

# Decoder
class Decoder(nn.Module):

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2048)(x)
        x = x.reshape(x.shape[0], 4, 4, 128)
        x = nn.ConvTranspose(features=128, kernel_size=(3, 3), strides=(2, 2))(x)
        x = nn.relu(x)
        x = nn.ConvTranspose(features=64, kernel_size=(3, 3), strides=(2, 2))(x)
        x = nn.relu(x)
        x = nn.ConvTranspose(features=32, kernel_size=(3, 3), strides=(2, 2))(x)
        x = nn.relu(x)
        x = nn.Conv(features=1, kernel_size=(3, 3), strides=1)(x)
        return x

In [15]:
# AutoEncoder
class AE(nn.Module):
    latents: int
    
    def setup(self):
        self.encoder = Encoder(self.latents)
        self.decoder = Decoder()

    def __call__(self, x):
        z = self.encoder(x)
        recon_x = self.decoder(z)
        return recon_x

    def generate(self, z):
        return nn.sigmoid(self.decoder(z))

ae = AE(EMBEDDING_DIM)
print(ae.tabulate(jax.random.PRNGKey(0), jnp.ones((1, 32, 32, 1)), console_kwargs={'width': 110}))


                                                  AE Summary                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path                  ┃ module        ┃ inputs              ┃ outputs             ┃ params                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│                       │ AE            │ float32[1,32,32,1]  │ float32[1,32,32,1]  │                        │
├───────────────────────┼───────────────┼─────────────────────┼─────────────────────┼────────────────────────┤
│ encoder               │ Encoder       │ float32[1,32,32,1]  │ float32[1,2]        │                        │
├───────────────────────┼───────────────┼─────────────────────┼─────────────────────┼────────────────────────┤
│ encoder/Conv_0        │ Conv          │ float32[1,32,32,1]  │ float32[1,16,16,32] │ bias: float32[32]      │


### Loss function

In [16]:
@jax.vmap
def binary_cross_entropy_with_logits(logits, label):
    logits = nn.log_sigmoid(logits)
    return -jnp.sum(labels * logits + (1. - labels) * jnp.log(-jnp.expm1(logits)))

## 3. Creating a `TrainState`

In [17]:
def create_train_state(model, rng, learning_rate):
    # Initialize the model
    params = model.init(rng, jnp.ones([1, 32, 32, 1]))['params']
    # Initialize the optimizer
    tx = optax.adam(LEARNING_RATE)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params,
        tx=tx
    )

## 4. Train step

In [18]:
@jax.jit
def train_step(state, batch, z_rng):
    def loss_fn(params):
        recon_x = state.apply_fn({'params': params}, batch['image'])
        loss = binary_cross_entropy_with_logits(recon_x, batch['image']).mean()
        return loss
    grad_fn = jax.grad(loss_fn)
    grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)
    return state

In [19]:
@jax.jit
def compute_metrics(*, state, batch):
    recon_x = state.apply_fn({'params': params}, batch['image'])
    loss = binary_cross_entropy_with_logits(recon_x, batch['image']).mean()
    return loss